In [12]:
import os
import rasterio
from rasterio.mask import mask
import geopandas as gpd
import pandas as pd
import numpy as np
from pyproj import Transformer, CRS

def create_show_progress(threshold=10):
    """
    创建一个进度回调函数，每当进度增加达到阈值时打印一次信息。
    """
    last_percentage = -threshold
    def Show_Progress(complete, message, unknown):
        nonlocal last_percentage
        percentage = int(complete * 100)
        if percentage >= last_percentage + threshold:
            last_percentage = percentage
            print("Progress: {}% - {}".format(percentage, message))
    return Show_Progress

def calculate_pixel_size_in_meters(raster_path, target_crs_str="ESRI:54009"):
    """
    利用中心像元的角点坐标，将其转换到目标投影后，
    计算像元的宽度、高度及面积（平行四边形假设）。
    """
    with rasterio.open(raster_path) as src:
        height, width = src.height, src.width
        center_row = height // 2
        center_col = width // 2

        # 获取中心像元的左上 (ul)、右上 (ur) 和左下 (ll) 角点坐标
        ul = src.xy(center_row, center_col, offset="ul")
        ur = src.xy(center_row, center_col, offset="ur")
        ll = src.xy(center_row, center_col, offset="ll")

        # 创建目标 CRS 对象
        target_crs = CRS.from_string(target_crs_str)
        transformer = Transformer.from_crs(src.crs, target_crs, always_xy=True)

        # 将角点坐标转换到目标投影
        ul_t = transformer.transform(ul[0], ul[1])
        ur_t = transformer.transform(ur[0], ur[1])
        ll_t = transformer.transform(ll[0], ll[1])

        # 计算转换后像元宽度和高度
        pixel_width = abs(ur_t[0] - ul_t[0])
        pixel_height = abs(ul_t[1] - ll_t[1])
        pixel_area = pixel_width * pixel_height

        return pixel_width, pixel_height, pixel_area

def remove_nodata(data, no_data):
    """
    移除数据中的 NoData 值、0 值以及 NaN 值。
    """
    if no_data is not None:
        mask_arr = (data == no_data) | (data == 0) | np.isnan(data)
    else:
        mask_arr = (data == 0) | np.isnan(data)
    return data[~mask_arr]

def zonal_statistics(raster_path, vector_path, output_path):
    """
    对每个分区（矢量多边形）进行栅格掩模和统计：
    1. 计算目标投影下的像元尺寸（以便后续计算像元面积）。
    2. 针对每个分区，利用 rasterio.mask.mask 提取该区域栅格数据，
       并计算有效像元个数、乘以像元面积和建筑覆盖率后得到建筑面积。
    3. 结果合并到矢量数据中，并保存输出。
    """
    try:
        # 打开栅格文件，获取 NoData 值和 CRS
        with rasterio.open(raster_path) as raster:
            no_data = raster.nodata
            raster_crs = raster.crs

        # 计算像元尺寸（目标投影下的尺寸，单位：米）
        pixel_size_x, pixel_size_y, pixel_area = calculate_pixel_size_in_meters(raster_path)
        print("Pixel width: {:.3f} m, Pixel height: {:.3f} m, Pixel area: {:.3f} m^2".format(pixel_size_x, pixel_size_y, pixel_area))

        # 读取矢量数据
        vector = gpd.read_file(vector_path)
        # 如果栅格不存在，则给出缺省值后退出
        if not os.path.exists(raster_path):
            print("Raster file {} not found. Setting ImpSurf, ISF to NoData.".format(raster_path))
            vector["ImpSurf"] = -9999
            vector["ISF"] = -9999

            vector.to_file(output_path)
            return

        # 保证矢量数据和栅格数据在相同 CRS 下
        if vector.crs != raster_crs:
            vector = vector.to_crs(raster_crs)

        # 初始化进度回调
        show_progress = create_show_progress(threshold=10)
        total_zones = vector.shape[0]
        results = []

        # 对每个分区（矢量多边形）进行处理
        for idx, row in vector.iterrows():
            geom = row['geometry']
            #print("Processing feature {}/{}".format(idx+1, total_zones))

            # 裁出一个小片栅格（掩模），默认只考虑像元中心落在多边形内部的像元
            with rasterio.open(raster_path) as raster:
                out_image, out_transform = mask(raster, [geom], crop=True, all_touched=False)
                data = out_image[0]
                valid_data = remove_nodata(data, no_data)

                # 如果存在有效数据，则计算建筑面积
                if valid_data.size > 0:
                    # 此处假设建筑覆盖率为有效像元平均值/100，
                    # 并以像元总面积乘以覆盖率得到建筑面积
                    count_val = valid_data.size
                    valid_building_area = count_val * pixel_area
                    area_val = valid_building_area
                    #print("Computed ImpSurf:", area_val)
                else:
                    area_val = -9999

                results.append({'ImpSurf': area_val})

            completed = (idx + 1) / float(total_zones)
            show_progress(completed, 'Processing zones', None)

        # 将统计结果合并到矢量数据中
        stats_df = pd.DataFrame(results)
        vector = vector.join(stats_df)

        # 这里CoverRatio和FAR的计算示例仅供参考，根据实际需求调整公式
        vector['ISF'] = np.where(vector['ImpSurf'] == -9999, -9999, vector['ImpSurf'] / vector['LandArea'])

        # 保存结果到输出文件
        vector.to_file(output_path)
        print("Zonal statistics saved to:", output_path)

    except Exception as e:
        print("Error calculating zonal statistics:", e)

#设置输入文件夹和输出文件路径
input_folder = r"D:\file\d_zonalst"

for place2 in os.listdir(input_folder):
    workspace = rf'D:\file\d_zonalst\{place2}'
    vector_path2 = os.path.join(workspace, f'{place2}_output_file(MEAN_BD_FAR_CD_CR_DistWB_DistGL_DistMT).shp')
    raster_path2 = rf'D:\file\d_builtup\{place2}\builtup_{place2}.tif'
    output_path2 = os.path.join(workspace, f'{place2}_output_file(+isf).shp')
    #
    # if os.path.exists(output_path2):
    #     print(rf'跳过{output_path2}:输出文件已经存在')
    #     continue
    zonal_statistics(raster_path2, vector_path2, output_path2)


Pixel width: 6.672 m, Pixel height: 8.994 m, Pixel area: 60.008 m^2
Progress: 0% - Processing zones
Progress: 10% - Processing zones
Progress: 20% - Processing zones
Progress: 30% - Processing zones
Progress: 40% - Processing zones
Progress: 50% - Processing zones
Progress: 60% - Processing zones
Progress: 70% - Processing zones
Progress: 80% - Processing zones
Progress: 90% - Processing zones
Progress: 100% - Processing zones
Zonal statistics saved to: D:\file\d_zonalst\0\0_output_file(+isf).shp
Pixel width: 6.672 m, Pixel height: 8.994 m, Pixel area: 60.008 m^2
Progress: 0% - Processing zones
Progress: 10% - Processing zones
Progress: 20% - Processing zones
Progress: 30% - Processing zones
Progress: 40% - Processing zones
Progress: 50% - Processing zones
Progress: 60% - Processing zones
Progress: 70% - Processing zones
Progress: 80% - Processing zones
Progress: 90% - Processing zones
Progress: 100% - Processing zones
Zonal statistics saved to: D:\file\d_zonalst\1\1_output_file(+isf).